### Read chat.db into a csv

In [12]:
import sqlite3
import pandas as pd

# Connect to the chat.db file
conn = sqlite3.connect('../data/chat.db')

# Execute a query to fetch the data
cursor = conn.cursor()
cursor.execute('''
    SELECT chat.ROWID, message.date, handle.id, chat.room_name, message.text, message.is_from_me
    FROM
    (message INNER JOIN chat_message_join ON message.ROWID = chat_message_join.message_id
                INNER JOIN chat ON chat.ROWID = chat_message_join.chat_id
                    INNER JOIN chat_handle_join ON chat.ROWID = chat_handle_join.chat_id
                        INNER JOIN handle ON handle.ROWID = chat_handle_join.handle_id)
    ORDER BY chat.ROWID, message.date
''')

# Fetch all the rows
rows = cursor.fetchall()

# Create a DataFrame from the fetched rows
df = pd.DataFrame(rows, columns=['ROWID', 'date', 'handle_id', 'room_name', 'text', 'is_from_me'])

df_with_room_name = df[df['room_name'].notna()]
df_without_room_name = df[df['room_name'].isna()]

df_grouped = df_with_room_name.groupby(['text', 'room_name', 'ROWID', 'date', 'is_from_me'], as_index=False).agg({'handle_id': ', '.join})

df_final = pd.concat([df_grouped, df_without_room_name])

# Sort the final DataFrame
df_sorted = df_final.sort_values(['ROWID', 'date'])

# Save the DataFrame as a CSV file
df_sorted.to_csv('../data/chat_grouped.csv', index=False)

### Sort by each chat instance